# Técnicas de Optimización en Spark

## Objetivos de Aprendizaje
- Entender cómo Spark ejecuta los trabajos
- Identificar y resolver cuellos de botella
- Aplicar técnicas de particionamiento y caching
- Optimizar joins y agregaciones

## Prerequisitos
- Todos los notebooks anteriores del módulo 06

## Tiempo Estimado
⏱️ 60 minutos

## Módulo AWS Academy Relacionado
📚 Módulo 9: Optimización de clusters EMR y jobs Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import time

spark = SparkSession.builder \
    .appName("SparkOptimization") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print(f"Spark {spark.version}")

---
# === SECCIÓN 1 ===
## 1. Modelo de Ejecución de Spark

### Explicación Conceptual
Spark divide el trabajo en:
- **Job**: Conjunto de tareas disparadas por una acción
- **Stage**: Conjunto de tareas que pueden ejecutarse en paralelo (sin shuffle)
- **Task**: Unidad mínima de trabajo en una partición

**Shuffle**: Redistribución de datos entre nodos. Es costoso porque implica escritura a disco y transferencia de red.

In [ ]:
# Generar datos de ejemplo
datos = [(i, f"producto_{i % 100}", i * 10.5, i % 10) 
         for i in range(100000)]

df = spark.createDataFrame(datos, ["id", "producto", "precio", "categoria"])

print(f"Registros: {df.count():,}")
print(f"Particiones iniciales: {df.rdd.getNumPartitions()}")

In [ ]:
# Ver el plan de ejecucion
# explain() muestra como Spark procesara la consulta

resultado = df.filter(F.col("categoria") == 5) \
    .groupBy("producto") \
    .agg(F.sum("precio").alias("total"))

print("Plan de ejecucion:")
resultado.explain()

# explain(True) muestra mas detalles
print("\nPlan detallado:")
resultado.explain(True)

---
# === SECCIÓN 2 ===
## 2. Particionamiento

### Explicación Conceptual
El **particionamiento** determina cómo se distribuyen los datos. Un buen particionamiento:
- Balancea la carga entre workers
- Minimiza shuffles
- Aprovecha el paralelismo

In [ ]:
# Repartition: Cambia el numero de particiones (causa shuffle)
df_8_particiones = df.repartition(8)
print(f"Despues de repartition(8): {df_8_particiones.rdd.getNumPartitions()} particiones")

# Coalesce: Reduce particiones SIN shuffle (mas eficiente)
df_4_particiones = df_8_particiones.coalesce(4)
print(f"Despues de coalesce(4): {df_4_particiones.rdd.getNumPartitions()} particiones")

In [ ]:
# Repartition por columna: Agrupa datos por clave
# Util antes de joins o groupBy

df_por_categoria = df.repartition(10, "categoria")
print(f"Particiones por categoria: {df_por_categoria.rdd.getNumPartitions()}")

# Ver distribucion de datos en particiones
distribucion = df_por_categoria.groupBy(F.spark_partition_id().alias("particion")) \
    .count() \
    .orderBy("particion")

print("\nDistribucion de registros por particion:")
distribucion.show()

---
# === SECCIÓN 3 ===
## 3. Caching y Persistencia

### Explicación Conceptual
**Cache** guarda un DataFrame en memoria para reutilizarlo sin recalcular.
- `cache()`: Guarda en memoria
- `persist()`: Permite elegir nivel de almacenamiento
- `unpersist()`: Libera la memoria

**Cuándo usar cache:**
- Cuando un DataFrame se usa múltiples veces
- Después de transformaciones costosas

In [ ]:
from pyspark import StorageLevel

# Crear DataFrame con transformaciones
df_transformado = df.filter(F.col("precio") > 100) \
    .withColumn("precio_iva", F.col("precio") * 1.16)

# Sin cache: Cada accion recalcula
start = time.time()
count1 = df_transformado.count()
count2 = df_transformado.filter(F.col("categoria") == 3).count()
tiempo_sin_cache = time.time() - start

# Con cache: Segunda accion usa datos en memoria
df_transformado.cache()
start = time.time()
count1 = df_transformado.count()  # Esto llena el cache
count2 = df_transformado.filter(F.col("categoria") == 3).count()  # Usa cache
tiempo_con_cache = time.time() - start

print(f"Sin cache: {tiempo_sin_cache:.3f}s")
print(f"Con cache: {tiempo_con_cache:.3f}s")

# Liberar memoria
df_transformado.unpersist()

In [ ]:
# Niveles de persistencia
print("Niveles de StorageLevel disponibles:")
print("  MEMORY_ONLY: Solo memoria (default de cache)")
print("  MEMORY_AND_DISK: Memoria, overflow a disco")
print("  DISK_ONLY: Solo disco")
print("  MEMORY_ONLY_SER: Serializado en memoria (menos RAM)")

# Ejemplo con persist
df_persistido = df.persist(StorageLevel.MEMORY_AND_DISK)
df_persistido.count()  # Materializa
df_persistido.unpersist()

---
# === SECCIÓN 4 ===
## 4. Optimización de Joins

### Explicación Conceptual
Los **joins** son operaciones costosas. Estrategias de optimización:
- **Broadcast Join**: Envía la tabla pequeña a todos los workers
- **Sort-Merge Join**: Para tablas grandes ordenadas por clave
- **Bucket Join**: Pre-particiona datos por clave de join

In [ ]:
# Crear tabla grande y pequeña
df_grande = spark.range(1000000).withColumn("valor", F.rand())
df_pequena = spark.createDataFrame(
    [(i, f"cat_{i}") for i in range(100)],
    ["id", "nombre"]
)

print(f"Tabla grande: {df_grande.count():,} filas")
print(f"Tabla pequeña: {df_pequena.count()} filas")

In [ ]:
# Broadcast Join: Para tablas pequenas (< 10MB por default)
# F.broadcast() fuerza broadcast de una tabla

join_broadcast = df_grande.join(
    F.broadcast(df_pequena),
    df_grande["id"] % 100 == df_pequena["id"]
)

print("Plan con Broadcast:")
join_broadcast.explain()

In [ ]:
# Comparar tiempos
# Join sin broadcast
start = time.time()
df_grande.join(df_pequena, df_grande["id"] % 100 == df_pequena["id"]).count()
tiempo_normal = time.time() - start

# Join con broadcast
start = time.time()
df_grande.join(F.broadcast(df_pequena), df_grande["id"] % 100 == df_pequena["id"]).count()
tiempo_broadcast = time.time() - start

print(f"Join normal: {tiempo_normal:.3f}s")
print(f"Broadcast join: {tiempo_broadcast:.3f}s")

---
# === SECCIÓN 5 ===
## 5. Evitar Anti-Patrones

### Explicación Conceptual
Algunos patrones de código causan problemas de rendimiento:
- **UDFs en Python**: Más lentas que funciones nativas
- **collect() en datos grandes**: Puede colapsar el driver
- **Shuffles innecesarios**: Evitar repartition sin necesidad

In [ ]:
# MAL: UDF de Python (lento)
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

@udf(returnType=DoubleType())
def calcular_iva_udf(precio):
    return precio * 1.16 if precio else None

# BIEN: Funcion nativa de Spark (rapido)
def calcular_iva_nativo(col):
    return col * 1.16

# Comparar tiempos
start = time.time()
df.withColumn("iva_udf", calcular_iva_udf("precio")).count()
tiempo_udf = time.time() - start

start = time.time()
df.withColumn("iva_nativo", calcular_iva_nativo(F.col("precio"))).count()
tiempo_nativo = time.time() - start

print(f"UDF Python: {tiempo_udf:.3f}s")
print(f"Funcion nativa: {tiempo_nativo:.3f}s")
print(f"Nativo es {tiempo_udf/tiempo_nativo:.1f}x mas rapido")

In [ ]:
# Buenas practicas
print("BUENAS PRÁCTICAS:")
print()
print("1. Filtrar temprano: Reduce datos antes de joins/aggregations")
print("2. Seleccionar columnas necesarias: No arrastrar columnas innecesarias")
print("3. Usar funciones nativas: Evitar UDFs cuando sea posible")
print("4. Cache con cuidado: Solo si se reutiliza el DataFrame")
print("5. Broadcast tablas pequeñas: < 10MB en joins")
print("6. Evitar collect(): Usar take(), show() o write()")
print("7. Particionar inteligentemente: Por columnas de join/groupBy")

---
# === EJERCICIOS PRÁCTICOS ===

### 🎯 Ejercicio O.1: Optimizar Consulta

Optimiza esta consulta ineficiente:

In [ ]:
# Consulta INEFICIENTE - Optimizala
def consulta_lenta():
    return df \
        .repartition(100) \
        .select("*") \
        .filter(F.col("categoria") == 5) \
        .groupBy("producto") \
        .agg(F.collect_list("precio").alias("precios"))

# TODO: Escribe version optimizada


### ✅ Solución Ejercicio O.1

In [ ]:
def consulta_optimizada():
    return df \
        .filter(F.col("categoria") == 5) \
        .select("producto", "precio") \
        .groupBy("producto") \
        .agg(
            F.sum("precio").alias("total"),
            F.count("*").alias("count")
        )

# Comparar
start = time.time()
consulta_lenta().count()
t1 = time.time() - start

start = time.time()
consulta_optimizada().count()
t2 = time.time() - start

print(f"Lenta: {t1:.3f}s")
print(f"Optimizada: {t2:.3f}s")

# Mejoras:
# 1. Filtrar ANTES de repartition
# 2. Seleccionar solo columnas necesarias
# 3. No repartition innecesario
# 4. Usar sum/count en lugar de collect_list

---
# === RESUMEN FINAL ===

## Resumen

### Conceptos Clave
- **Particionamiento**: `repartition()` vs `coalesce()`, particionar por clave
- **Caching**: `cache()`, `persist()`, `unpersist()` para reutilizar datos
- **Broadcast Join**: Para tablas pequeñas
- **Evitar**: UDFs, collect() en datos grandes, shuffles innecesarios
- **Mejores prácticas**: Filtrar temprano, seleccionar columnas, usar funciones nativas

### Conexión con AWS
- **EMR**: Configuración de clusters para optimizar Spark
- **Glue**: Tiene optimizaciones automáticas (pushdown, partitioning)
- **S3**: Particionamiento de datos en S3 mejora lecturas

### Siguiente Paso
¡Felicidades! Has completado el módulo de Spark Processing. Continúa con:
- `07_ml_data_preparation/` para preparación de datos ML